In [1]:
# default_exp examine
%load_ext autoreload
%autoreload 2

# examine


> API details.

In [2]:
#hide
from nbdev.showdoc import *

In [3]:
#export
from fastai.layers import flatten_model
from fastai.vision.all import *

![pipeline](images/graphic11.PNG)

In [10]:
#export
def visualize_layers(fn, learn, nrows=5, ncols=5):
    "Visualize how an image is transformed as it goes through the layers in the model"
    img_list = []; size_list = []
    head_layer = flatten_model(learn.model[0])
    print(f'Number of Layers: {len(head_layer)}')
    for layer in head_layer:
        layer_name = str(layer)
        custom_hook = hook_output(layer)
        learn.predict(fn)
        activation = custom_hook.stored[0][0]
        layimg = TensorImage(activation)
        sh = layimg.shape
        img_list.append(layimg)
        size_list.append(sh)
    show_images(img_list, titles=size_list, nrows=nrows, ncols=ncols);

In [12]:
#export
def view_layers(image, layer:int, layers, **kwargs):
    lsl = []
    custom_hook = hook_outputs(layers)
    pre = learn.predict(image)
    _,s,w,h = custom_hook.stored[layer].shape
    for i in range(s):
        acts = custom_hook.stored[layer][0][i]
        lsl.append(acts)
    print(f'Layer: {layers[layer]}')
    print(f'Number of activations: {s}')
    print(f'Image shape: {w, h}')
    show_images(lsl, nrows=1)

In [13]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 02_explore.ipynb.
Converted 03_preprocessing.ipynb.
Converted 04_pipeline.ipynb.
Converted 05_train.ipynb.
Converted 06_examine.ipynb.
Converted 10_wearable.ipynb.
Converted 90_tutorial.ipynb.
Converted index.ipynb.
